In [20]:
import sys
import numpy as np

In [80]:
def push_relabel(C, F, source, sink):
    n = len(C) # C is the capacity matrix    
    
    height = [0] * n # height of nodes
    excess = [0] * n # flow into node minus flow from node
    neighbour = [0] * n # neighbours seen since last relabel
    nodelist = [i for i in range(n) if i != source and i != sink] # node queue
    
    def push(u, v):
        push_flow = min(excess[u], C[u][v] - F[u][v])
        F[u][v] += push_flow
        F[v][u] -= push_flow
        excess[u] -= push_flow
        excess[v] += push_flow
        
    def relabel(u):
        # find smallest new height making a push possible, if such a push is possible
        min_height = sys.maxsize
        for v in range(n):
            if C[u][v] - F[u][v] > 0:
                min_height = min(min_height, height[v])
                height[u] = min_height + 1
    
    def discharge(u):
        while excess[u] > 0:
            if neighbour[u] < n: # check next neighbour
                v = neighbour[u]
                if C[u][v] - F[u][v] > 0 and height[u] > height[v]:
                    push(u, v)
                else:
                    neighbour[u] += 1
            else: # all neighbours of u have been checked, must relabel u
                relabel(u)
                neighbour[u] = 0
    
    height[source] = n # longest path from source to sink is less than n long
    excess[source] = sys.maxsize # send as much flow as possible to neighbours of source
    for v in range(n):
        push(source, v)
    
    i = 0
    while i < len(nodelist):
        u = nodelist[i]
        old_height = height[u]
        discharge(u)
        if height[u] > old_height:
            nodelist.insert(0, nodelist.pop(i)) # start from front of nodelist
            i = 0
        else:
            i += 1
    return sum(F[source])

In [78]:
def load_data(path):
    with open(path, "rt") as f:
        i = 1
        idx = 1
        while True:
            if i <= 3:
                f.readline()
            else:
                line = f.readline()
                if not line:
                    break
                split_line = line.split()                
                M = int(split_line[0])
                N = int(split_line[1])
                print("M: %d, N: %d" %(M, N))
                row_sum = f.readline().split()
                row_sum = list(map(int, row_sum))
                col_sum = f.readline().split()
                col_sum = list(map(int, col_sum))
                matrix_size = M * N + 2
                C = np.zeros((matrix_size, matrix_size), dtype=int)
                F = [[0] * matrix_size for _ in range(matrix_size)] # residual capacity from u to v is C[u][v] - F[u][v]
                for i in range(M):
                    for j in range(N):
                        C[i][M + j] = 1 # C[i][j]
                for i in range(M):
                    C[matrix_size - 2][i] = row_sum[i] # C[s][i]
                for j in range(N):
                    C[M + j][matrix_size - 1] = col_sum[j] # C[j][t]
                
                max_flow = push_relabel(C, F, matrix_size - 2, matrix_size - 1)
                if max_flow != sum(row_sum):
                    print("Can't find matrix!")
                    idx += 1
                else:
                    matrix = np.zeros((M, N), dtype=int)

                    for i in range(M):
                        for j in range(N):
                            matrix[i][j] = F[i][M + j]
                    result_row_sum = np.sum(matrix, axis=1)
                    result_col_sum = np.sum(matrix, axis=0)

                    print("max flow: %d" %max_flow)
                    print("The %dst matrix is:" %idx)
                    print(matrix)
                    print("The sum of rows are:")
                    print(result_row_sum)
                    print("The sum of cols are:")
                    print(result_col_sum)
                    idx += 1
            i += 1

In [79]:
load_data("./problem9.data")

M: 10, N: 10
max flow: 55
The 1st matrix is:
[[1 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 1 1 1 1 1 1]
 [1 0 1 0 0 1 1 1 1 1]
 [0 1 1 1 1 1 0 0 0 1]
 [0 1 1 0 0 1 0 0 0 0]
 [0 1 1 1 1 0 0 0 0 1]
 [0 1 1 1 1 1 1 0 0 1]
 [1 1 1 1 1 1 1 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]]
The sum of rows are:
[5 5 7 7 6 3 5 7 7 3]
The sum of cols are:
[6 6 7 4 5 6 6 4 4 7]
M: 13, N: 14
max flow: 94
The 2st matrix is:
[[1 0 0 0 0 1 1 1 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 1 1 1 1]
 [1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 1 0 0 1 0 0 1]
 [1 0 0 1 0 0 0 1 1 0 1 1 0 1]
 [1 1 0 1 0 0 0 1 1 0 1 1 0 1]
 [0 1 1 1 0 0 1 1 1 0 1 1 1 0]
 [0 1 1 1 1 0 1 1 1 1 1 0 0 0]
 [0 1 1 1 1 0 1 1 1 0 1 0 0 0]
 [0 1 1 1 1 0 1 0 0 0 0 0 0 1]
 [0 1 1 1 1 1 1 1 1 1 0 0 0 1]
 [1 1 1 1 1 0 0 0 0 0 0 0 0 0]]
The sum of rows are:
[10  5  7  6  4  7  8  9  9  8  6 10  5]
The sum of cols are:
[ 8  7  6  8  5  2  6  8  8  5 10  7  5  9]
M: 120, N: 110
max flow: 6684
The 3st matrix is:
[[0 1 1 ..., 